<a href="https://colab.research.google.com/github/lmaciuga/rock-paper-scissors/blob/master/Rock_paper_scisorrs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rock, paper, scissors deep learning

#### Install libraries

In [ ]:
pip install -q tensorflow tensorflow-datasets

#### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

#### data set list

In [ ]:
tfds.list_builders()

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'arc',
 'bair_robot_pushing_small',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'cos_e',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eraser_multi_rc',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flic',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'gr

#### Dataset info

In [ ]:
builder = tfds.builder('rock_paper_scissors')
info = builder.info
info

####Prepare data

In [ ]:
ds_train = tfds.load('rock_paper_scissors', split='train')
ds_test = tfds.load('rock_paper_scissors', split='test')

#### Examples

In [ ]:
fig = tfds.show_examples(info, ds_train)

#### Data preparation

In [ ]:
train_images = np.array([image['image'].numpy()[:,:,0] for image in ds_train])
train_labels = np.array([label['label'].numpy() for label in ds_train])

test_images = np.array([image['image'].numpy()[:,:,0] for image in ds_test])
test_labels = np.array([label['label'].numpy() for label in ds_test])

In [ ]:
train_images = train_images.reshape(2520, 300, 300, 1).astype('float32')
test_images = test_images.reshape(372, 300, 300, 1).astype('float32')

train_images /= 255
test_images /= 255

#### Train network (basic approach)

In [ ]:
model = keras.Sequential([
                          keras.layers.Flatten(),
                          keras.layers.Dense(512, activation='relu'),
                          keras.layers.Dense(256, activation='relu'),
                          keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32)

In [ ]:
model.evaluate(test_images, test_labels)

####Train network (convolutional approach)

In [ ]:
model = keras.Sequential([
                          keras.layers.Conv2D(64, 3, activation='relu', input_shape=(300,300,1)),
                          keras.layers.Conv2D(32, 3, activation='relu'),
                          keras.layers.Flatten(),
                          keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32)

In [ ]:
model.evaluate(test_images, test_labels)

#### A better convolutional network

In [ ]:
model = keras.Sequential([
                          keras.layers.AveragePooling2D(6,3, input_shape=(300,300,1)),
                          keras.layers.Conv2D(64, 3, activation='relu'),
                          keras.layers.Conv2D(32, 3, activation='relu'),
                          keras.layers.MaxPool2D(2,2),
                          keras.layers.Dropout(0.5),
                          keras.layers.Flatten(),
                          keras.layers.Dense(128, activation='relu'),
                          keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=32)

In [ ]:
model.evaluate(test_images, test_labels)

#### Hyperparameter turning

In [ ]:
pip install keras-tuner

In [ ]:
from kerastuner.tuners import RandomSearch

def build_model(hp):
  model = keras.Sequential()

  model.add(keras.layers.AveragePooling2D(6,3, input_shape=(300,300,1)))

  for i in range(hp.Int('Conv layers', min_value=0, max_value=3)):
    model.add(keras.layers.Conv2D(hp.Choice(f'layer_{i}_filters', [16, 32, 64]), 3, activation='relu'))
  
  model.add(keras.layers.MaxPool2D(2,2))
  model.add(keras.layers.Dropout(0.5))
  model.add(keras.layers.Flatten())
  
  model.add(keras.layers.Dense(hp.Choice('Dense layer', [64, 128, 256, 512, 1024]), activation='relu'))
  
  model.add(keras.layers.Dense(3, activation='softmax'))

  model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

  return model

tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5)

tuner.search(train_images, train_labels, validation_data=(test_images, test_labels), epochs=10, batch_size=32)

In [ ]:
best_model = tuner.get_best_models()[0]

In [ ]:
best_model.evaluate(test_images, test_labels)

In [ ]:
best_model.summary()

In [ ]:
tuner.results_summary()

#### Save & Load model

In [ ]:
best_model.save('./my_model')

In [ ]:
loaded_model = keras.models.load_model('./my_model')

In [ ]:
loaded_model.evaluate(test_images, test_labels)

#### Plot image from numpy array

In [ ]:
train_images[0].shape

In [ ]:
image = train_images[0].reshape(300, 300)
plt.imshow(image, cmap='Greys_r')

#### Plot colored image

In [ ]:
rgb_images = np.array([example['image'].numpy() for example in ds_train.take(1)])
rgb_image = rgb_images[0]

plt.imshow(rgb_image)

#### Convert PNG/JPG images to numpy

In [ ]:
import imageio

image = imageio.imread('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Persons_died_due_to_coronavirus_COVID-19_per_capita_in_Europe.svg/1280px-Persons_died_due_to_coronavirus_COVID-19_per_capita_in_Europe.svg.png')

plt.imshow(image)

image_np = np.asarray(image)
type(image_np)
image_np.shape